# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from pprint import pprint
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model
import requests
import json

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'wine-quality-automl'

experiment=Experiment(ws, experiment_name)

# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "wine-quality"
description_text = "Wine Quality Dataset for Udacity Course 3"

if key in ws.datasets.keys(): 
    found = True
    input_data = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    url_white = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"
    url_red = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    white_data = TabularDatasetFactory.from_delimited_files(url_white, separator=";")
    red_data = TabularDatasetFactory.from_delimited_files(url_red, separator=";")
    features, target = clean_data(white_data, red_data)
    features.loc[:,"quality"]=target
    ds = ws.get_default_datastore()
    input_data = TabularDatasetFactory.register_pandas_dataframe(dataframe=features, target=ds, name=key,
                                                             description=description_text)


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
Method register_pandas_dataframe: This is an experimental method, and may change at any time.<br/>For more information, see https://aka.ms/azuremlexperimental.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/477edf41-d631-47ed-980d-98f0f829424b/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [4]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority',
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
automl_settings = {
    "n_cross_validations": 3,
    "enable_early_stopping": True,
    "primary_metric": "AUC_weighted",
    "max_concurrent_iterations": 5,
    
}
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    compute_target=compute_target,
    training_data=input_data,
    label_column_name="quality",
    **automl_settings)

In [6]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, tag={'type':'automl'})

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=====

        61   StandardScalerWrapper XGBoostClassifier        0:01:13       0.8239    0.8673
        62   MinMaxScaler LogisticRegression                0:00:57       0.7645    0.8673
        63   StandardScalerWrapper XGBoostClassifier        0:00:56       0.8547    0.8673
        65   SparseNormalizer XGBoostClassifier             0:00:25          nan    0.8673
        64                                                  0:00:20          nan    0.8673
        66                                                  0:00:48          nan    0.8673
        67    VotingEnsemble                                0:02:04       0.8756    0.8756
        68    StackEnsemble                                 0:02:25       0.8742    0.8756


{'runId': 'AutoML_d0df9350-7860-45c3-94c4-6a4cc557bdce',
 'target': 'aml-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-12T16:50:25.415885Z',
 'endTimeUtc': '2021-02-12T17:25:31.325546Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'aml-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"wine-quality-automl","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-138710","workspace_name":"quick-starts-ws-138710","region":"southcentralus","compute_target":"aml-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_cl

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
assert (automl_run.get_status() == "Completed")
best_ml_run, best_model = automl_run.get_output()


def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators':
                       list(e[0] for e in step[1].estimators),
                   'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()
            
print_model(best_model)


Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['49', '34', '47', '31', '63', '27', '52', '38', '28'],
 'weights': [0.26666666666666666,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667,
             0.06666666666666667,
             0.2,
             0.13333333333333333,
             0.06666666666666667,
             0.06666666666666667]}

49 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': False,
 'with_std': False}

49 - xgboostclassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,


In [16]:
best_ml_run.download_file("./outputs/model.pkl")
best_ml_run.download_file("outputs/conda_env_v_1_0_0.yml", "conda_env.yml")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [10]:
registered_model = best_ml_run.register_model(model_name="wine-quality-model", model_path="./outputs/model.pkl")

service_name = 'wine-quality-classification'
inference_config = InferenceConfig(entry_script='score.py', environment=best_ml_run.get_environment())
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)
service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)
print(service.scoring_uri)
primary_key, _ = service.get_keys()

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://b4cb163d-6032-4d56-b881-e4486c9570e5.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [13]:
data = {"data":
         [{
             "fixed acidity":7.90000,
             "volatile acidity":0.52000,
             "citric acid":0.26000, 
             "residual sugar":2.2000,
             "chlorides":0.07900,
             "free sulfur dioxide":14.0000,
             "total sulfur dioxide":38.0000,
             "density":0.99675,
             "pH":3.31000,
             "sulphates":0.62000,
             "alcohol":10.20000,
             "is_red_wine": 1
         },
         {
             "fixed acidity":6.80,
             "volatile acidity":0.26000,
             "citric acid":0.32000, 
             "residual sugar":5.2000,
             "chlorides":0.04300,
             "free sulfur dioxide":34.0000,
             "total sulfur dioxide":134.00000,
             "density":0.99374,
             "pH":3.18000,
             "sulphates":0.47000,
             "alcohol":10.40000,
             "is_red_wine": 0
         }]
     }
header = {'Content-Type': 'application/json'}
header["Authorization"] = f"Bearer {primary_key}"
json_data = json.dumps(data)

resp = requests.post(service.scoring_uri, json_data, headers=header)
print(resp.json())

{"result": ["MEDIUM", "MEDIUM"]}


TODO: In the cell below, print the logs of the web service and delete the service

In [12]:
print(service.get_logs())
#service.delete()

2021-02-12T17:43:53,427041452+00:00 - rsyslog/run 
2021-02-12T17:43:53,429665614+00:00 - nginx/run 
2021-02-12T17:43:53,431376320+00:00 - gunicorn/run 
2021-02-12T17:43:53,437663709+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd